This jupyter notebook contains the computations used in Section 4 of the article "On the codimension-two cohomology of $\operatorname{SL}_n(\mathbb{Z})$" by Benjamin Brück, Jeremy Miller, Peter Patzt, Robin J. Sroka, and Jennifer C. H. Wilson. The references in this notebook refer to this article, hereafter [BMPSW], in the following version: ARXIV LINK

The notebook can be used to list all the necessary isomorphism types of the complexes $Q^m_n(\vec v_1,\vec v_2,\vec v_3;\vec w)$, defined in [BMPSW, Definition 3.50], and show that these complexes are 3-connected.

To build the complexes and compute their homology, gudhi and SageMath are used. For installation instructions, see 

    https://doc.sagemath.org/html/en/installation/index.html
    
    https://gudhi.inria.fr
    
In case of questions, please contact

    benjamin.brueck@math.ethz.ch

In [1]:
import numpy as np
from scipy.special import comb
import gudhi

from sage.homology.homology_group import HomologyGroup

In [2]:
# Import functions from the auxiliary files in this directory
import linear_algebra as lin_alg
import simplex_constructor as sim_constr
import complex_constructor as com_constr
import complex_toolbox as com_tools
import homology_fibres as hom_fib

# List all isomorphism types

The isomorphism type of the complex $Q^m_n(\vec v_1,\vec v_2,\vec v_3;\vec w)$ only depends on the $R$-intervals of the last entries of $\vec v_1$, $\vec v_2$, $\vec v_3$, $\vec v_1+\vec v_2$, $\vec v_1+\vec v_2+ \vec v_3$, $\vec v_1+\vec v_3$ (see [BMPSW, Proposition 4.6]). In what follows, we list all combinations of $R$-intervals that can occur among these last entries and then find a representative for each such type (see [BMPSW, Corollary 4.8]).

In [3]:
# Compute all possible combinations using [BMPSW, Lemma 4.3].
# Using that v1+v3 = (v1)+(v3) = (-v2)+(v1+v2+v3)
list_iso_types = hom_fib.iso_types(3,[[[0,1]],[[2,3]],[[0,2],[-1,4]]])
print('Initial number of isomorphism types that exist: '+str(len(list_iso_types))+'\n')
print('These are:')
for isotype in list_iso_types:
    print(isotype)

Initial number of isomorphism types that exist: 147

These are:
[-1, -1, -1, -3, -5, -3]
[-1, -1, -1, -3, -4, -3]
[-1, -1, -1, -3, -3, -3]
[-1, -1, -1, -3, -3, -2]
[-1, -1, -1, -3, -3, -1]
[-1, -1, -1, -2, -3, -3]
[-1, -1, -1, -2, -3, -2]
[-1, -1, -1, -2, -3, -1]
[-1, -1, -1, -1, -3, -3]
[-1, -1, -1, -1, -3, -2]
[-1, -1, -1, -1, -3, -1]
[-1, -1, -1, -1, -2, -1]
[-1, -1, -1, -1, -1, -1]
[-1, -1, 0, -3, -3, -1]
[-1, -1, 0, -2, -2, -1]
[-1, -1, 0, -1, -1, -1]
[-1, -1, 1, -3, -3, -1]
[-1, -1, 1, -3, -2, -1]
[-1, -1, 1, -3, -1, 0]
[-1, -1, 1, -3, -1, 1]
[-1, -1, 1, -3, -1, -1]
[-1, -1, 1, -2, -1, 0]
[-1, -1, 1, -2, -1, 1]
[-1, -1, 1, -2, -1, -1]
[-1, -1, 1, -1, 0, 1]
[-1, -1, 1, -1, 1, 1]
[-1, -1, 1, -1, -1, 0]
[-1, -1, 1, -1, -1, 1]
[-1, -1, 1, -1, -1, -1]
[-1, 0, -1, -1, -3, -3]
[-1, 0, -1, -1, -2, -2]
[-1, 0, -1, -1, -1, -1]
[-1, 0, 0, -1, -1, -1]
[-1, 0, 1, -1, 0, 0]
[-1, 0, 1, -1, 1, 1]
[-1, 0, 1, -1, -1, -1]
[-1, 1, -1, 0, -1, -3]
[-1, 1, -1, 0, -1, -2]
[-1, 1, -1, 0, -1, -1]
[-1, 1, 

## Remove symmetries from the list
As shown in [BMPSW, Lemma 4.7], certain combinations of $R$-intervals $(r_1,r_2,r_3,r_{12},r_{123},r_{13})$ lead to the same isomorphism type of the complex $Q(r_1,r_2,r_3,r_{12},r_{123},r_{13})$. In what follows, we remove these duplicates from list_iso_types. 

In [4]:
# Remove the symmetry around 0; afterwards, iso_types contains either type of -type, 
# but not both [BMPSW, Lemma 4.7.ii])
helper_set = set()
for j in range(len(list_iso_types)):
    current_type = list_iso_types.pop(0)
    if not tuple(current_type) in helper_set:
        list_iso_types.append(current_type) # representative of the iso class
        # add all isomorphic elements to helper_set
        helper_set.add(tuple(current_type))
        helper_set.add(tuple(-np.array(current_type)))
print('Isomorphism types that are left: '+str(len(list_iso_types)))

Isomorphism types that are left: 74


In [5]:
#Remove the symmetry between v2 and v3 [BMPSW, Lemma 4.7.i]
helper_set = set()
for j in range(len(list_iso_types)):
    current_type = list_iso_types.pop(0)
    if not tuple(current_type) in helper_set:
        list_iso_types.append(current_type) # representative of the iso class
        # add all isomorphic elements to helper_set
        helper_set.add(tuple(current_type))
        new_order = [0,2,1,5,4,3] # order after changing v2 and v3
        reordered_type = [current_type[i] for i in new_order]
        helper_set.add(tuple(reordered_type))
print('Isomorphism types that are left: '+str(len(list_iso_types)))

Isomorphism types that are left: 48


## Export the list of isomorphism types
The following was used to produce [BMPSW, Table 1], which lists the 48 remaining representatives for isomorphism types of $Q(r_1,r_2,r_3,r_{12},r_{123},r_{13})$. It can be directly entered to a latex tabular environment.

In [6]:
iso_types_for_printing = []
for poss in list_iso_types:
    iso_types_for_printing.append(poss)
iso_types_for_printing.sort()
for row in iso_types_for_printing:
    row_string = str(row)
    row_string = row_string.replace("[", "")
    row_string = row_string.replace("]", "")
    row_string = row_string.replace(",", " &")
    row_string = row_string+" \\\\"
    print(row_string)

-1 & -1 & -1 & -3 & -5 & -3 \\
-1 & -1 & -1 & -3 & -4 & -3 \\
-1 & -1 & -1 & -3 & -3 & -3 \\
-1 & -1 & -1 & -3 & -3 & -2 \\
-1 & -1 & -1 & -3 & -3 & -1 \\
-1 & -1 & -1 & -2 & -3 & -2 \\
-1 & -1 & -1 & -2 & -3 & -1 \\
-1 & -1 & -1 & -1 & -3 & -1 \\
-1 & -1 & -1 & -1 & -2 & -1 \\
-1 & -1 & -1 & -1 & -1 & -1 \\
-1 & -1 & 0 & -3 & -3 & -1 \\
-1 & -1 & 0 & -2 & -2 & -1 \\
-1 & -1 & 0 & -1 & -1 & -1 \\
-1 & -1 & 1 & -3 & -3 & -1 \\
-1 & -1 & 1 & -3 & -2 & -1 \\
-1 & -1 & 1 & -3 & -1 & -1 \\
-1 & -1 & 1 & -3 & -1 & 0 \\
-1 & -1 & 1 & -3 & -1 & 1 \\
-1 & -1 & 1 & -2 & -1 & -1 \\
-1 & -1 & 1 & -2 & -1 & 0 \\
-1 & -1 & 1 & -2 & -1 & 1 \\
-1 & -1 & 1 & -1 & -1 & -1 \\
-1 & -1 & 1 & -1 & -1 & 0 \\
-1 & -1 & 1 & -1 & -1 & 1 \\
-1 & -1 & 1 & -1 & 0 & 1 \\
-1 & -1 & 1 & -1 & 1 & 1 \\
-1 & 0 & 0 & -1 & -1 & -1 \\
-1 & 0 & 1 & -1 & -1 & -1 \\
-1 & 0 & 1 & -1 & 0 & 0 \\
-1 & 0 & 1 & -1 & 1 & 1 \\
-1 & 1 & 1 & -1 & -1 & -1 \\
-1 & 1 & 1 & -1 & 0 & -1 \\
-1 & 1 & 1 & -1 & 1 & -1 \\
-1 & 1 & 1 & 0 & 1 & -1

# Find representatives for the isomorphims types
In what follows, we find for each combination of $R$-intervals in list_iso_types a 4-tuple (z1,z2,z3,R) such that the numbers (z1,z2,z3,z1+z2,z1+z2+z3, z1+z3) have exactly this combination of $R$-intervals. (These will be the last entries of the vectors $\vec v_1, \vec v_2,\vec v_3,\vec w$ that are used to construct $Q^m_n(\vec v_1,\vec v_2,\vec v_3;\vec w)$ below.)

In [7]:
# Now find a representative for each isomorphism type

representatives = []
iso_candidates = hom_fib.potential_last_entries(4) # search for representatives in this list

for z1,z2,z3,R in iso_candidates:
    iso_type = [
        hom_fib.Rint(z1,R),
        hom_fib.Rint(z2,R),
        hom_fib.Rint(z3,R),
        hom_fib.Rint(z1+z2,R),
        hom_fib.Rint(z1+z2+z3,R),
        hom_fib.Rint(z1+z3,R)
    ]
    if iso_type in list_iso_types:
        # then we don't have a representatives for this iso_type yet
        list_iso_types.remove(iso_type)
        representatives.append([[z1,z2,z3,R],iso_type])
if len(list_iso_types) == 0:
    print('Found a representative for every isomorphism type.')

Found a representative for every isomorphism type.


In [8]:
for representative,iso_type in representatives:
    print(str(representative)+' represents the isomorphism type '+str(iso_type))

[-3, -3, -3, 4] represents the isomorphism type [-1, -1, -1, -3, -5, -3]
[-3, -3, -2, 4] represents the isomorphism type [-1, -1, -1, -3, -4, -3]
[-3, -3, -1, 4] represents the isomorphism type [-1, -1, -1, -3, -3, -2]
[-3, -3, 0, 4] represents the isomorphism type [-1, -1, 0, -3, -3, -1]
[-3, -3, 1, 4] represents the isomorphism type [-1, -1, 1, -3, -3, -1]
[-3, -3, 2, 4] represents the isomorphism type [-1, -1, 1, -3, -2, -1]
[-3, -3, 3, 4] represents the isomorphism type [-1, -1, 1, -3, -1, 0]
[-3, -2, -2, 4] represents the isomorphism type [-1, -1, -1, -3, -3, -3]
[-3, -2, 2, 4] represents the isomorphism type [-1, -1, 1, -3, -1, -1]
[-3, -1, -1, 4] represents the isomorphism type [-1, -1, -1, -2, -3, -2]
[-3, -1, 0, 4] represents the isomorphism type [-1, -1, 0, -2, -2, -1]
[-3, -1, 1, 4] represents the isomorphism type [-1, -1, 1, -2, -1, -1]
[-3, -1, 3, 4] represents the isomorphism type [-1, -1, 1, -2, -1, 0]
[-3, 0, 0, 4] represents the isomorphism type [-1, 0, 0, -1, -1, -1]


# Simple connectivity
We now show that for every complex on the list of isomorphism types, the number of missing 2-simplices is so low that it implies that the complex is simply connected [BMPSW, Lemma 4.16].

In [9]:
for representative in representatives:
    
    last_entries = representative[0]
    # Complete these last entries to vectors that form a basis of Z^4:
    matrix = hom_fib.complete_to_invertible(last_entries)

    v1 = matrix[0]
    v2 = matrix[1]
    v3 = matrix[2]
    w = matrix[3]
    
    # Construct Q_4^0(v1,v2,v3,w):
    vectors_defining_Q = [v1,v2,v3,v1+v2,v1+v2+v3,v1+v3]
    vertices_of_Q = hom_fib.vertices_fibre(vectors_defining_Q,w)
    Q = com_constr.link_BAA(vertices_of_Q,[w])   
        
    num_vertices = Q.num_vertices()
    
    # First check whether the entire 1-skeleton is there
    num_one_simplices_Q = com_tools.number_simplices_dimensions(Q)[1]
    num_possible_one_simplices = comb(num_vertices,2)
    if not num_one_simplices_Q == num_possible_one_simplices:
        print('Error: There is a complex that has a pair of vertices that does not form an edge.')
        break
    
    num_two_simplices_Q = com_tools.number_simplices_dimensions(Q)[2]
    num_possible_two_simplices = comb(num_vertices,3)
    num_non_two_simplices = num_possible_two_simplices - num_two_simplices_Q

    if num_non_two_simplices < num_vertices - 2:
        print('The complex is simply connected.')
    else:
        print('The argument does not show that the complex is simply connected.')

The complex is simply connected.
The complex is simply connected.
The complex is simply connected.
The complex is simply connected.
The complex is simply connected.
The complex is simply connected.
The complex is simply connected.
The complex is simply connected.
The complex is simply connected.
The complex is simply connected.
The complex is simply connected.
The complex is simply connected.
The complex is simply connected.
The complex is simply connected.
The complex is simply connected.
The complex is simply connected.
The complex is simply connected.
The complex is simply connected.
The complex is simply connected.
The complex is simply connected.
The complex is simply connected.
The complex is simply connected.
The complex is simply connected.
The complex is simply connected.
The complex is simply connected.
The complex is simply connected.
The complex is simply connected.
The complex is simply connected.
The complex is simply connected.
The complex is simply connected.
The comple

# Homology calculations
We now use Sage to compute the integral homology of these complexes.
https://doc.sagemath.org/html/en/reference/topology/sage/topology/simplicial_complex.html

These computations show that all the complexes are 3-acyclic, which is the last ingredient needed for the proof of [BMPSW, Theorem 3.51].

In [10]:
trivial_homology_group = HomologyGroup(0,ZZ)
three_acyclic = True

for representative in representatives:
    
    last_entries = representative[0]
    iso_type = representative[1]
    # Complete these last entries to vectors that form a basis of Z^4:
    matrix = hom_fib.complete_to_invertible(last_entries)
    
    v1 = matrix[0]
    v2 = matrix[1]
    v3 = matrix[2]
    w = matrix[3]
    
    # Construct Q_4^0(v1,v2,v3,w):
    vectors_defining_Q = [v1,v2,v3,v1+v2,v1+v2+v3,v1+v3]
    vertices_of_Q = hom_fib.vertices_fibre(vectors_defining_Q,w)
    Q = com_constr.link_BAA(vertices_of_Q,[w])   
    
    # Create a SimplicialComplex in Sage from this and compute its homology:
    Q_sage = SimplicialComplex([])
    for simplex in Q.get_simplices():
        Q_sage.add_face(simplex[0])
    homology = Q_sage.homology(base_ring = ZZ)
    print('The reduced integral homology of Q'+str(iso_type)+' is given by')
    print(homology)
    
    # Check whether the complex is 3-acyclic:
    for n in range(4):
        if not homology[n] == trivial_homology_group:
            three_acyclic = False
            print('Found a complex that is not 3-acyclic.\n')
            break
    if three_acyclic:
        print('This is trivial in degrees 0,1,2,3.\n')
    else:
        break

if three_acyclic:
    print('The complexes are all 3-acyclic.')

The reduced integral homology of Q[-1, -1, -1, -3, -5, -3] is given by
{0: 0, 1: 0, 2: 0, 3: 0, 4: Z^110}
This is trivial in degrees 0,1,2,3.

The reduced integral homology of Q[-1, -1, -1, -3, -4, -3] is given by
{0: 0, 1: 0, 2: 0, 3: 0, 4: Z^63}
This is trivial in degrees 0,1,2,3.

The reduced integral homology of Q[-1, -1, -1, -3, -3, -2] is given by
{0: 0, 1: 0, 2: 0, 3: 0, 4: Z^63}
This is trivial in degrees 0,1,2,3.

The reduced integral homology of Q[-1, -1, 0, -3, -3, -1] is given by
{0: 0, 1: 0, 2: 0, 3: 0, 4: Z^63}
This is trivial in degrees 0,1,2,3.

The reduced integral homology of Q[-1, -1, 1, -3, -3, -1] is given by
{0: 0, 1: 0, 2: 0, 3: 0, 4: Z^110}
This is trivial in degrees 0,1,2,3.

The reduced integral homology of Q[-1, -1, 1, -3, -2, -1] is given by
{0: 0, 1: 0, 2: 0, 3: 0, 4: Z^63}
This is trivial in degrees 0,1,2,3.

The reduced integral homology of Q[-1, -1, 1, -3, -1, 0] is given by
{0: 0, 1: 0, 2: 0, 3: 0, 4: Z^63}
This is trivial in degrees 0,1,2,3.

The reduc